<a href="https://colab.research.google.com/github/hdakhli/mlops-2024/blob/main/31_llm_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install langchain-core langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Tester un LLM avec une question hyper spécifique

In [30]:
# Utiliser langchain avec le model llama
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q2_K.gguf"
)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Vous êtes un assistant utile. Vous êtes là pour répondre à mes questions {input}.",
        )
    ]
)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32

In [31]:
chain.invoke({"Qu'est-ce qu'un sans effet ?"})


llama_print_timings:        load time =    2818.30 ms
llama_print_timings:      sample time =      42.61 ms /   141 runs   (    0.30 ms per token,  3309.47 tokens per second)
llama_print_timings: prompt eval time =    4534.73 ms /    37 tokens (  122.56 ms per token,     8.16 tokens per second)
llama_print_timings:        eval time =   15292.97 ms /   140 runs   (  109.24 ms per token,     9.15 tokens per second)
llama_print_timings:       total time =   20399.96 ms /   177 tokens


'\n\nVous êtes un assistant utile, c\'est-à-dire un modèle de langage basé sur l\'apprentissage automatique qui est là pour vous aider à résoudre des problèmes. Voici les réponses aux questions que vous avez formulées :\n\n* Un effet est une chose qui peut causer un résultat ou un impact. Par exemple, "Le nouveau médicament a un grand effet sur la santé" ou "L\'incendie a eu un grand effet sur les maisons environnantes".\nI hope this helps! Let me know if you have any other questions.'

### Avec du RAG

In [32]:
!pip install langchain-chroma llama-index-embeddings-langchain llama-index sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 4.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.6.5
    Uninstalling nltk-3.6.5:
      Successfully uninstalled nltk-3.6.5
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [33]:
# define embedding function using sentence-transformers/all-mpnet-base-v2 model from HuggingFace
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# load documents using PyPDFLoader
PDF_PATH = "datasets/contrat.pdf"
loader = PyPDFLoader(PDF_PATH)
pages = loader.load_and_split()

# create vectorstore and store pages into it
vectorstore = Chroma.from_documents(pages, embedding=embeddings, persist_directory="./db")

/Users/a443xq/Documents/GitHub/api-model/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)


In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms.llamacpp import LlamaCpp
from langchain_core.runnables import RunnablePassthrough

retriever = vectorstore.as_retriever()

model_path = "llama-2-7b-chat.Q2_K.gguf"
llm = LlamaCpp(model_path=model_path, verbose=False, n_ctx=2048, f16_kv=True)

template = """Answer the question based only on the following context:

{context}
if the information is not in {context}, JUST say: I dont know !
You should always respond in french
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

In [39]:
chain.invoke("Qu'est-ce qu'un sans effet ?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'Answer: Un contrat est considéré comme avoir "without effect" (Sans Effet in French) cuando :\n* La date d\'effet de ce contrat est dans le futur et que vous décidez de vous rétracter.\n* Le véhicule à assurer n\'est finalement pas acheté.\n* Vous vendez le véhicule avant que le contrat ne prenne effet.\n* Vous souscrivez chez nous dans le cadre d\'une loi Hamon, mais la date d\'effet de votre contrat que vous nous demandes ne nous permet pas de résilier votre contrat chez votre assureur actuel.'

In [ ]:
# Enrichir la base vectorielle chroma DB par les cours que vous avez eu cette année
# Tester !